<a href="https://colab.research.google.com/github/Navjotkhatri/Productionization_of_ML_Systems_in_Travel_Industry/blob/main/REST_API_for_Flight_Price_Regression_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **2. REST API for Regression Model:**

**Develop a REST API using Flask to serve the flight price prediction model, enabling real-time price predictions.**

In [1]:
!ngrok authtoken "2kEiDvlDvDmlSrjFdidw1UnLQmx_32guShAgCSJ9W3gqsYXPA"

/bin/bash: line 1: ngrok: command not found
